<a href="https://colab.research.google.com/github/TermProject-BigData-2021-2/preprocess/blob/master/%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0_processing_hyunju.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **colab mount**

이 코드를 돌리기 위해선...
자기 구글 드라이브 > "빅데이터"라는 폴더 생성 > mbti_1.csv 파일(캐글에서 제공해주는 파일임)을 여기다가 갖다 놓기

path: drive/MyDrive/빅데이터/mbti_1.csv

그리고 아래 코드 돌리면 잘 돌아갑니다~

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **data 불러오기**

In [55]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm
import re
import nltk 
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from wordcloud import WordCloud
from nltk.corpus import stopwords 


# 데이터 불러오기
data = pd.read_csv("drive/MyDrive/빅데이터/mbti_1.csv")
data.head()

#타입에 따른 개수
data['type'].value_counts()

# 데이터 정보
data.info()

px.pie(data,names='type',title='Personality type',hole=0.3)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8675 entries, 0 to 8674
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    8675 non-null   object
 1   posts   8675 non-null   object
dtypes: object(2)
memory usage: 135.7+ KB


# **Lemmatizer 정의**

In [56]:
class Lemmatizer(object):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
    def __call__(self, sentence):
        return [self.lemmatizer.lemmatize(word) for word in sentence.split() if len(word)>2]

# **data 전처리**

<추가한 컬럼 내용>
1. 이모티콘 개수 
2. 이미지 개수
3. 느낌표 개수
4. mbti 개수
5. url 개수

<데이터 정화>
- http 뺌
- lower 화 함
- alpha bet만 남김
- lemmatazing
- stemming


In [59]:

# 이모티콘 정의
smiley = [';)', ':)', ':-)', '|-)', '|-D', ' :->', '\'-)', ';->', ':*)', ';-)', 'B-)', '8-]', ':-]', 'xD', ':^D', '^^', 'XD']
unhappy = [':(', ':-(' , ';(', ':\'(', ':-c', ':-C', ':-<', ':-X', ':-x', ':-@', ':-&', ':-r', ':-V', ':@', '\-o', ':-I', 'T_T']
another = [':O', ':-o', '8-O', '8-|', ':-T', ':/', ':P', ':-p', ':3', '8-|', ':-\\', ':~/']
imoticon = smiley + unhappy + another

#이미지 정의
basic = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.svg', '.ai', '.psd', '.tiff']
more = ['.mp4', '.cr2', '.srw', '.nrw', '.tga']
image = basic + more

#필요한 배열
exclamation_count_column = []
mbti_count_column = []
clean = []
imo_count = []
img_count = []
final = []

#stop word
cachedStopWords = stopwords.words("english")

#lemmatizer
lemmatizer = WordNetLemmatizer()

# 데이터 정제 
for sentence in tqdm(data.posts):

    # 느낌표 개수 column
    exclamation_count = sentence.count("!")
    exclamation_count_column.append(exclamation_count)

    # mbti라는 단어의 개수
    sentence_lower = sentence.lower()
    mbti_count = sum(x in {'mbti',"estj","estp","esfj","esfp","entp","entj","enfp","enfj","istj","istp","isfj","isfp","intp","intj","infp","infj"} for x in nltk.wordpunct_tokenize(sentence_lower))
    mbti_count_column.append(mbti_count)

    # 이모티콘 개수
    sentence_simple_html = re.sub('(http|ftp|https|uhttp|$uhttp)://', 'URL', sentence)
    num =0
    for i in imoticon:
        num += sentence_simple_html.count(i)
    imo_count.append(num)

    # 이미지 개수
    text = sentence_simple_html.split('|||')
    text = ' '.join(text)
    text = text.split(' ')
    onlyURL = []
    for item in text:
        if 'URL' in item:
            onlyURL.append(item)
    num = 0
    for url in onlyURL:
        for tname in image:
            if tname in url[-5:]:
                num += 1
                break
    img_count.append(num)

    # 소문자 치환
    sentence = sentence.lower()
    # 마지막에 http 없애기
    sentence = re.sub('https?://[^\s<>"]+|www\.[^\s<>"]+',' ',sentence)
    # 특수문자, 숫자 모두 제거
    sentence = re.sub('[^a-z]',' ',sentence)
    #stop word 제거
    #sentence = " ".join([lemmatizer.lemmatize(w) for w in sentence.split(' ') if w not in cachedStopWords])
    # 앞뒤 자르기
    sentence = sentence.strip()
    # 공백 여러개면 하나로 줄이기
    sentence = ' '.join(sentence.split())


    clean.append(sentence)



100%|██████████| 8675/8675 [01:33<00:00, 92.41it/s]


# **전처리 바탕으로 데이터 완성**

final : [post 내용 , 이미지 개수 , 이모티콘 개수 , 느낌표 개수 , MBTI 개수(MBTI + 유형)]

In [54]:
# print(exclamation_count_column)
# print(mbti_count_column)
# print(imo_count)
# print(img_count)

vectorizer=TfidfVectorizer( max_features=5000,stop_words='english',tokenizer=Lemmatizer())
vectorizer.fit(clean)

feature_names=vectorizer.get_feature_names()
wc=WordCloud(max_words=400)
wc.generate(' '.join(word for word in feature_names[500:3500] ))
plt.figure(figsize=(20,15))
plt.axis('off')
plt.imshow(wc)

#final = list(zip(clean, img_count, imo_count, exclamation_count_column, mbti_count_column))
#print(final[0])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning:

The parameter 'token_pattern' will not be used since 'tokenizer' is not None'

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning:

Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'wa'] not in stop_words.



['abandon', 'abandoned', 'ability', 'able', 'abnormal', 'abortion', 'abroad', 'absence', 'absent', 'absolute', 'absolutely', 'absorbed', 'abstract', 'absurd', 'abuse', 'abused', 'abusive', 'academic', 'accent', 'accept', 'acceptable', 'acceptance', 'accepted', 'accepting', 'access', 'accident', 'accidentally', 'accomplish', 'accomplished', 'accomplishment', 'according', 'account', 'accounting', 'accuracy', 'accurate', 'accurately', 'accused', 'ace', 'achieve', 'achieved', 'achievement', 'achieving', 'acid', 'acknowledge', 'acquaintance', 'act', 'acted', 'acting', 'action', 'active', 'actively', 'activity', 'actor', 'actress', 'actual', 'actually', 'adam', 'adapt', 'add', 'added', 'addict', 'addicted', 'addiction', 'addictive', 'adding', 'addition', 'additional', 'address', 'adhd', 'adjust', 'admirable', 'admire', 'admit', 'admitted', 'admittedly', 'admitting', 'adopt', 'adopted', 'adorable', 'adore', 'adrenaline', 'adult', 'advance', 'advanced', 'advantage', 'adventure', 'adventurous',

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.

